In [1]:
config_path = "test\\Config.xlsx"
OUTPUT_PATH = "test\\Processed"
data_path = "test\\data\\2020 Novartis SOW_Capmatinib US _V19.xlsx"
# data_path = "Check\\data"
log_path = "test\\Logs"
import pandas as pd

In [4]:
from ExcelParser.MakeDF import CleanDFByConfig
from io import BytesIO
import os
import traceback


def process_sheet(data_path, sheet_name, in_mem_file, cdf, filename):
    df_sow, status, actual_sheet_name = None, None, None
    try:
        df_sow = cdf.get_df(data_path, bytesIO=in_mem_file, sheet_name=sheet_name, excel_table=True)
        actual_sheet_name = df_sow.iloc[0].SheetName or sheet_name
    except Exception as e:
        traceback.print_exc()
        df_sow, status =  {
                   "status": "Error",
                   "filename": filename,
                   "error": e,
                   "sheet": actual_sheet_name
               }, None
    else:
        status = {
                   "status": "Success",
                   "filename": filename,
                   "error": "",
                   "sheet": actual_sheet_name
               }
    return {
            "status":status,
            "df":df_sow
    }


def process_sow(config_path, data_path):
    filename = data_path[data_path.rfind("\\") + 1:]
    cdf = CleanDFByConfig(config_path)
    a = {}
    with open(data_path, "rb") as f:
        in_mem_file = BytesIO(f.read())
        a["sow desc"] = process_sheet(data_path, "sow desc", in_mem_file, cdf, filename)
        a["sow tactical"] = process_sheet(data_path, "sow tactical", in_mem_file, cdf, filename)
    return a


In [5]:
p = process_sow(config_path, data_path)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


{'project name|project desc|project assump': {'coordinate': (22, 3), 'cell_value': 'Project Name'}}
Directly read dataframe
Extract Direct Table
Header is : 22 and sheet name is 1. SOW Description
{'functional hour|department|location': {'coordinate': (28, 2), 'cell_value': 'Location'}}
Directly read dataframe
Extract Direct Table
Header is : 28 and sheet name is 2. SOW Tactical & Staff Plan


In [7]:
for _, v in p.items():
    print(v["status"])

{'status': 'Success', 'filename': '2020 Novartis SOW_Capmatinib US _V19.xlsx', 'error': '', 'sheet': '1. SOW Description'}
{'status': 'Success', 'filename': '2020 Novartis SOW_Capmatinib US _V19.xlsx', 'error': '', 'sheet': '2. SOW Tactical & Staff Plan'}


In [6]:
p['sow tactical']["df"].iloc[0]

Functional Hourly Rate                                291
Total Fees                                         160050
Title                         SVP Client Service Director
Currency                                            (USD)
Name                                     Bethany Sardinha
Tier                                               Tier 3
Total Hours                                           550
Agency                                    DDB Health (US)
Time Based On FTE Hours                          0.305556
Department                             Account_Management
Location                                               US
ID                         2020NovartisSOWCapmatinibUSV19
row_number                                             29
SOW FTE Hours                                        1800
Novartis Division                                Oncology
Lead Agency                               DDB Health (US)
Lead Agency Currency                              $ (USD)
Discipline    

In [ ]:
k = "43468"

In [ ]:
k.isnumeric(), k.isdigit()

In [ ]:
p['sow tactical']["df"].iloc[0]["SheetName"]

In [ ]:
from dateutil.parser import parse
origin_date = parse("1900-01-01")
d = str(43468).strip()
origin_date + pd.DateOffset(days=int(d))

In [ ]:
def clean_date(field_series, *args, **kwargs):
    origin_date = parse("1900-01-01")

    def parse_to_date(x):
        d = str(x).strip()
        try:
            if d.isnumeric():
                return origin_date + pd.DateOffset(days=int(d))
            elif d == 'None':
                return parse("1900-01-01")
            else:
                k = parse(d)
                return k if len(d) >= 5 else parse(f"2020-{d}-01")
        except:
            return parse("1900-01-01")

    return pd.to_datetime(field_series.apply(parse_to_date), errors="coerce")